In [5]:
%%bash

# pip install --user pandas kfp kubeflow-metadata
# pip install Ipython --user
# pip install config --user
# pip install kubernetes --user
# pip install sklearn

In [11]:
'data_test'.find('jest')

-1

In [3]:
from ipython.display import display_html
def restart_kernel():
    display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)
    
restart_kernel()

ModuleNotFoundError: No module named 'ipython'

In [4]:
import kfp.dsl as dsl
import kfp
import pandas as pd
import kfp.gcp as gcp
from kfp import components
import json
from string import Template
from config import Config
import os
import kubernetes
import yaml
import sklearn

In [78]:
with open("./train_image/train_template.yaml") as yml_in:
    b = yaml.safe_load(yml_in)
print(b)
# json.dumps(b)
train_temp = Template(f"""{str(b)}""")


{'apiVersion': 'batch/v1', 'kind': 'Job', 'metadata': {'name': '{{.Trial}}', 'namespace': '{{.Namespace}}'}, 'spec': {'template': {'spec': {'restartPolicy': 'Never', 'containers': [{'name': '{{.Trial}}', 'image': '$image', 'imagePullPolicy': 'Always', 'command': ['python /opt/model.py --tfmode=local {{- with .Hyperparameters}} {{- range .}} {{.Name}}={{.Value}} {{- end}} {{- end}}']}]}}}}


In [82]:
trainjson = train_temp.substitute({'image': 'iamrock'})
trainjson

"{'apiVersion': 'batch/v1', 'kind': 'Job', 'metadata': {'name': '{{.Trial}}', 'namespace': '{{.Namespace}}'}, 'spec': {'template': {'spec': {'restartPolicy': 'Never', 'containers': [{'name': '{{.Trial}}', 'image': 'iamrock', 'imagePullPolicy': 'Always', 'command': ['python /opt/model.py --tfmode=local {{- with .Hyperparameters}} {{- range .}} {{.Name}}={{.Value}} {{- end}} {{- end}}']}]}}}}"

In [83]:
json.dumps(trainjson)

'"{\'apiVersion\': \'batch/v1\', \'kind\': \'Job\', \'metadata\': {\'name\': \'{{.Trial}}\', \'namespace\': \'{{.Namespace}}\'}, \'spec\': {\'template\': {\'spec\': {\'restartPolicy\': \'Never\', \'containers\': [{\'name\': \'{{.Trial}}\', \'image\': \'iamrock\', \'imagePullPolicy\': \'Always\', \'command\': [\'python /opt/model.py --tfmode=local {{- with .Hyperparameters}} {{- range .}} {{.Name}}={{.Value}} {{- end}} {{- end}}\']}]}}}}"'

In [40]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

/secret/gcp/user-gcp-sa.json


In [39]:
dir(kubernetes.client)

['AdmissionregistrationApi',
 'AdmissionregistrationV1beta1Api',
 'AdmissionregistrationV1beta1ServiceReference',
 'AdmissionregistrationV1beta1WebhookClientConfig',
 'ApiClient',
 'ApiextensionsApi',
 'ApiextensionsV1beta1Api',
 'ApiextensionsV1beta1ServiceReference',
 'ApiextensionsV1beta1WebhookClientConfig',
 'ApiregistrationApi',
 'ApiregistrationV1Api',
 'ApiregistrationV1beta1Api',
 'ApiregistrationV1beta1ServiceReference',
 'ApisApi',
 'AppsApi',
 'AppsV1Api',
 'AppsV1beta1Api',
 'AppsV1beta1Deployment',
 'AppsV1beta1DeploymentCondition',
 'AppsV1beta1DeploymentList',
 'AppsV1beta1DeploymentRollback',
 'AppsV1beta1DeploymentSpec',
 'AppsV1beta1DeploymentStatus',
 'AppsV1beta1DeploymentStrategy',
 'AppsV1beta1RollbackConfig',
 'AppsV1beta1RollingUpdateDeployment',
 'AppsV1beta1Scale',
 'AppsV1beta1ScaleSpec',
 'AppsV1beta1ScaleStatus',
 'AppsV1beta2Api',
 'AuditregistrationApi',
 'AuditregistrationV1alpha1Api',
 'AuthenticationApi',
 'AuthenticationV1Api',
 'AuthenticationV1beta

In [17]:
def convert_result(result) -> str:
    import json
    hyperparameters = json.loads(result)
    args = []
    print(type(hyperparameters))
    for key, value in hyperparameters.items():
        args.append("{0}={1}".format(key, value))
#         print(param)
        
    return " ".join(args)
x = '{"name":"John", "age":30, "city":"New York", "value": "none yet"}'
convert_result(x)       

<class 'dict'>


'name=John age=30 city=New York value=none yet'

In [33]:
PROJECT_ID = os.environ.get('PROJECT_ID')


None


In [80]:
@dsl.pipeline(
    name="Police-Shootings-Justification-Prediction",
    description="A pipeline to train and serve police shootings justification in the United States"
)

def police_shootings_pipeline(
    name="police-shootings-{{workflow.uid}}",
    katib_namespace="kubeflow-olawunmi1602",
    goal=0.9,
    max_trial_count=3,
    parallel_trial_count=3,
    training_steps=5,
    training_image="gcr.io/kubeflow-292422/police-shootings-train-sklearn:latest",
    training_namespace="kubeflow",
    serving_name="police-shootings-server",
    serving_namespace="kubeflow",
    serving_export_dir="gs://<BUCKET_NAME>/export",
    transformer_image="gcr.io/kubeflow-292422/police-shootings-processing:latest"
):
    
    objectiveConfig = {
        "type": "maximize",
        "goal": goal,
        "objectiveMetricName": "auc_score",
        "additionalMetricNames": ["loss", "accuracy"]
    }
    
    algorithmConfig = {
        "algorithmName": "random"
    }
    
    metricsCollectorSpec = {
        "collector": {
            "kind": "StdOut"
        }
    }
    
    parameters = [
        {"name": "--tf-learning-rate", "parameter-type": "double", "feasibleSpace": {"min": "0.001", "max": "0.005"}}
    ]
    
    train_template = b
    train_json = train_template.substitute({'image': str(training_image)})
    
    trialTemplate = {
        "goTemplate": {
            "rawTemplate": json.dumps(train_json)
        }
    }
    
    katib_experiment_launcher_op = components.load_component_from_url(
        'https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/katib-launcher/component.yaml'
    )
    
    katib_op = katib_experiment_launcher_op(
        experiment_name=name,
        experiment_namespace=katib_namespace,
        parallel_trial_count=parallel_trial_count,
        max_trial_count=max_trial_count,
        objective=str(objectiveConfig),
        algorithm=str(algorithmConfig),
        trial_template=str(trialTemplate),
        parameters=str(parameters),
        metrics_collector=str(metricsCollectorSpec),
        delete_finished_experiment=False
    )
    
    convert_op = components.func_to_container_op(convert_result)
    op2 = convert_op(katib.output)